#  Data mining of DNA sequences submitted by Peruvian institutions to public genetic databases

This Jupyter notebook will adress the code used in this study. For more information please contact:

Pedro Romero, pedro.romero@upch.pe
Camila Castillo-Vilcahuaman, camila.castillo.v@upch.pe (owner of the repository)


First of all, let's see which files we have available for this analysis. 

**05/2020**: As of now, I'm unable to upload the `journal_and_organism` file due to its size. I will be adressing this issue in the future. Meanwhile, I'll show here which code we used for extracting data from the `Nucleotide` database.

**04/2021**: I decided to upload the `journal_and_organism` file in my Drive. Please, download it from here: https://drive.google.com/drive/folders/1jVXrafIG3gtJGZvley01aavu4Q3GFPzQ?usp=sharing

In [1]:
%%bash
ls
#Hi, I'm leaving the csvkit package installation code here too. I used it for the last bits of the code, so you may require it to explore all this work.
#pip install csvkit

1.txt                   Carta de presentación.docx
200804_bioproject.xml   data_orgn_type_inst
archaea.txt             draft_scripts
BinderBash-master/      grepjournal
BinderBash-master.zip   journal_and_organism
bioproject_peru.xml     list
BOLD_abad               list_commas
bold_Amazonia           mining_peruvian_sequences.ipynb
BOLD_amazonia           organizationNames
BOLD_antunez            organizationNames_2
bold_centro             organizationNames_3
BOLD_centro             PATRIC_genome.csv
bold_data_species.tsv   Pedro_bioproject20200408.xml
bold_data_species.txt   Pedro_bioproject_peru.xml
bold_data_species.xlsx  peptides.txt
BOLD_imarpe             prueba
BOLD_senasa             prueba2
BOLD_ucayali            prueba_total
BOLD_unalm              tax/
BOLD_unalmherbario      taxonom/
BOLD_unfv               test.csv
BOLD_unmsm              Universidad_Cientifica_del_Sur
BOLD_unmsmmuseo         used_queries_per_inst_nucleotide.txt
BOLD_upch


## Nucleotide

`Nucleotide` was the biggest database we had to analyze. In this case, I used a server to download all the data related to the term: "Peru" using Entrez Direct (available at the NCBI page):

`esearch -db nucleotide -query "Peru" | efetch -format gb > peru.gb`

Once we had the `peru.gb` file, we ran an `awk` script for fetching all the data related to journals and organisms in the `peru.gb` file.

` awk '{ if($1 ~ /JOURNAL/ || $1 ~ /ORGANISM/){ ban = 1; } if($1 ~ /REFERENCE/ || $1 ~ /COMMENT/ || $1 ~ /PUBMED/ || $1 ~ /FEATURES/ || $1 ~ /AUTHORS/|| $1 ~ /REMARK/ || $1 ~ /TITLE/){ ban = 0; } if(ban == 1){ print $0; } }' peru.gb > journal_and_organism `

The output was named `journal_and_organism`
For checking out how many organisms we had in our `journal_and_organism` data, we used the `grep` command.

`grep -c "ORGANISM" journal_and_organism`

This gave us 817 694 records associated with the term "Peru" in our `journal_and_organism` file. However, we knew that this number could be an overstimation, because the term "Peru" is not only related to sequences uploaded from this country. Thus, we decided to perform an analysis including all institutions.

The Nucleotide format breaks paragraphs, thus making it difficult to search for institutions using their complete name. Also, uploaders have the freedom to adress their sequences as they need, which means that some institutions have variations in their names. For example, the Instituto Nacional de Salud was also found in this database using the english translation. This meant that we had to search carefully for query words. The file `used_queries_per_inst_nucleotide.txt` contains all the query words used for this analysis. 

In [3]:
%%bash
head used_queries_per_inst_nucleotide.txt

Se aplicaron los siguientes queries para cada institución (recordar que el código awk vuelve todo el archivo a mayúsculas):

Universidad Nacional de San Agustín

SAINT AUGUSTINE	1
SAN AGUSTIN	51
SAINT AGUSTIN	3

*No uso UNSA porque en algunas líneas aparece que "Universidad" y "UNSA" están en la misma línea. Parece que sería redundante.



To make the analysis easier, we made a list of all query words used:

In [4]:
%%bash
head list

SAINT AUGUSTINE
SAN AGUSTIN
SAINT AGUSTIN
UNAS
NACIONAL MAYOR
UNMSM
AGRARIAN INNOVATION
INNOVACION AGRARIA
NACIONAL DE SALUD
FARVET


For the first part, we decided to perform manually a search of each query word, to see how many times an institution appeared in the `journal_and_organism` data. For example, here we used the query word "SAN MARCOS":

In [9]:
%%bash
awk -v j=0 -v col=0 -v total_col=0 '{
if($1 ~ /JOURNAL/){
if(j == 1){
if(col == 1){
total_col = total_col + 1;
col = 0;
}
} else {
j = 1;
}
}
if(toupper($0) ~ /SAN MARCOS/){
col = 1;
}
}END{
print "Numero total de revistas con palabra clave: " total_col;
}' journal_and_organism

Numero total de revistas con palabra clave: 4951


Once we had a first scan of all the institutions and how many times their names appeared in our database, we performed a search to determine which organisms came from this institutions. For this, we used a `while` loop:

In [13]:
%%bash
while read p; do awk -v orgn="" -v p="$p" '{ if ($1 ~ /ORGANISM/) { ban = 1; orgn = $2 " " $3 } if (toupper($0) ~ p) { if (ban == 1) { print orgn; ban = 0 } } }' journal_and_organism > j_and_orgn_"$p"; done < list
cat j_and_orgn_* > j_and_orgn_total

After this, we concatenated all the output data using the `cat` command. The output file was called `total` (I prefer having all these files in a directory, so I created one and then moved all the files generated by the previous script). After this, we used a `sort` and a `uniq` command to count all unique species in our data.

In [24]:
%%bash
sort j_and_orgn_total | uniq -c > j_and_orgn_cont_total
head j_and_orgn_cont_total

      3 Acarichthys heckelii
      1 Acarospora cf.
      3 Acestrorhynchus falcatus
      6 Acestrorhynchus falcirostris
      3 Acestrorhynchus nasutus
      2 Achromobacter denitrificans
      4 Achromobacter sp.
      2 Achromobacter xylosoxidans
      1 Acidiphilium acidophilum
    212 Acidithiobacillus ferrivorans


For checking out information about taxa, we used this command:

In [22]:
%%bash
while read p; do awk -v orgn="" -v p="$p" -v orgn2="" -v orgn3="" '{ if ($1 ~ /ORGANISM/) { ban = 1; getline orgn; getline orgn2; getline orgn3 } if (toupper($0) ~ p) { if (ban == 1) { print orgn; print orgn2; print orgn3; ban = 0 } } }' journal_and_organism > j_and_orgn_"$p"; done < list
cat j_and_orgn_* > total
head total

            Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales;
            Rhizobiaceae; Rhizobium/Agrobacterium group; Rhizobium.
  JOURNAL   Unpublished
            Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales;
            Rhizobiaceae; Rhizobium/Agrobacterium group; Rhizobium.
  JOURNAL   Unpublished
            Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales;
            Rhizobiaceae; Rhizobium/Agrobacterium group; Rhizobium.
  JOURNAL   Unpublished
            Bacteria; Proteobacteria; Alphaproteobacteria; Rhizobiales;


This allowed us to check out which domains and classes were the most represented in peruvian sequences. We also used the `cat` command to create a file containing all the information from all the generated files. For checking out which taxa were present in this `total` file, we used the `grep -c` command.

In [23]:
%%bash
grep -c "Pasteurella multocida" total

1782


I suggest to search the species and taxonomic groups reported in our paper using this `grep` command. We did some manual curation to the data obtained in our files, so these should be checked out if used. Some grep combinations allow you to create lists and then to paste them somewhere else where its easier to manipulate tables.

If you have run this code, you will notice that certain numbers are lost during the process. We assume this is because of how heterogenous the metadata is. Institution names could be repeated twice or could be badly written, and, when in search for organism data, we detected some organisms which had no complete taxonomic classification. 

## Bioproject

For the BioProject data, we used the following command to extract the info required for this survey: 
`esearch -db bioproject -query "peru" | efetch -mode xml`

There wasn't much data to explore here. We used the following commands to navigate through our Bioproject data.

In [ ]:
%%bash
#Extract organizationNames:

awk -v ban=0 '{
if($0 ~ /<Organization/) {
ban = 1;
}
if(ban == 1){
print $0;
}
if($0 ~ /<\/Organization>/) {
ban = 0;
}
}' Pedro_bioproject20200408.xml > organizationNames_2

##Extract genome status:

awk -v ban=0 '{
if($0 ~ /<ProjectTypeSubmission>/ || $0 ~ /<Organization/){
ban = 1;
}
if(ban == 1){
print $0;
}
if($0 ~ /<\/Organization>/ || $0 ~ /<\/Organism>/){
ban = 0;
}
}' bioproject_peru.xml > data_orgn_type_inst

awk -v targ="" '{
if ($0 ~ /<Target/) {
targ = $0;
}
if ($0 ~ /<OrganismName/) {
targ = targ";"$0
}
if ($0 ~ /<Supergroup/) {
targ = targ";"$0
}
if ($0 ~ /<Organization/) {
org = 1
}
if($0 ~ /<Name/) {
if (org == 1) {
if ($0 ~ /Universidad Cientifica del Sur/) {
print targ;
targ = "";
}
}
}
if ($0 ~ /<\/Organization>/) {
org = 0;
}
}' data_orgn_type_inst > Universidad_Cientifica_del_Sur

##Extract names and count them:

grep -o 'Universidad Cientifica del Sur' organizationNames | wc -l
grep -o 'FARVET S.A.C.' organizationNames | wc -l
grep -o 'INCABIOTEC SAC' organizationNames | wc -l
grep -o 'Instituto Nacional de Salud - Peru' organizationNames | wc -l
grep -o 'International Potato Center' organizationNames | wc -l
grep -o 'Universidad Nacional Agraria La Molina, Peru' organizationNames | wc -l
grep -o 'Universidad Cientifica del Peru' organizationNames | wc -l
grep -o 'Universidad Nacional de San Antonio Abad' organizationNames | wc -l
grep -o 'Universidad Peruana Cayetano Heredia' organizationNames | wc -l

grep -o 'Universidad Peruana Cayetano Heredia' bioproject_peru.xml | wc -l
grep -o 'Peruvian National Institute of Health' organizationNames_2

## PATRIC

The PATRIC database showed to be more homogenous. PATRIC data can be downloaded in a `.csv` format, which makes it easier to analyze. PATRIC data can be found in this repository.

In [5]:
%%bash
head PATRIC_genome.csv

Genome ID,Genome Name,Organism Name,NCBI Taxon ID,Genome Status,Strain,Serovar,Biovar,Pathovar,MLST,Other Typing,Culture Collection,Type Strain,Completion Date,Publication,BioProject Accession,BioSample Accession,Assembly Accession,SRA Accession,GenBank Accessions,RefSeq Accessions,Sequencing Centers,Sequencing Status,Sequencing Platform,Sequencing Depth,Assembly Method,Chromosomes,Plasmids,Contigs,Genome Length,GC Content,PATRIC CDS,RefSeq CDS,Isolation Site,Isolation Source,Isolation Comments,Collection Date,Isolation Country,Geographic Location,Latitude,Longitude,Altitude,Depth,Other Environmental,Host Name,Host Gender,Host Age,Host Health,Body Sample Site,Body Sample Subsite,Other Clinical,AntiMicrobial Resistance,AntiMicrobial Resistance Evidence,Gram Stain,Cell Shape,Motility,Sporulation,Temperature Range,Optimal Temperature,Salinity,Oxygen Requirement,Habitat,Disease,Comments,Additional Metadata,Coarse Consistency,Fine Consistency,Checkm Completeness,Checkm Contamination,Genome 

To count how many institutions have uploaded information to the PATRIC database, we used the `grep` command.

In [6]:
%%bash
grep -c "Universidad Nacional Mayor de San Marcos" PATRIC_genome.csv

10


To extract all organisms uploaded in the PATRIC database, we used an `awk` command:

In [7]:
%%bash
cat PATRIC_genome.csv | cut -f2 -d , | awk '{print $1,$2}' | sort | uniq -c

      3 "Acidithiobacillus ferrivorans
      4 "Acinetobacter baumannii
      1 "Aeromonas sobria
      1 "Aeromonas veronii
      1 "Arcobacter sp.
      2 "Arthrospira sp.
      1 "Avibacterium paragallinarum
      1 "Bacillus thuringiensis
     14 "Bartonella bacilliformis
      1 "Bradyrhizobium icense
      1 "Bradyrhizobium paxllaeri
      1 "Bradyrhizobium sp.
      1 "Burkholderia pseudomallei
      1 "Candidatus Bartonella
      2 "Candidatus Marinimicrobia
      2 "Candidatus Poribacteria
      1 "Candidatus Thioglobus
      4 "Candidatus Tokpelaia
      2 "Candidatus Woesearchaeota
      2 "Dehalococcoidia bacterium
      2 "Enterobacter hormaechei
      1 "Enterobacter soli
      1 "Enterococcus faecium
     11 "Escherichia coli
      1 "Flavobacterium sp.
      1 "Fulvimarina sp.
     10 "Gammaproteobacteria bacterium
      1 "Gemmatimonadetes bacterium
      1 Genome Name
      1 "Gimesia sp.
      1 "Gordonia sp.
      1 "Gramella sp.
      2 "Haliea sp.
      1 "Halobac

To extract all organisms per institution, I had to download the `csvgrep` package.

**05/2020**: I'm unsure if I can download that package to this enviroment. I'll try this in the future.

In [8]:
%%bash
csvgrep -c 22 -m "Instituto Nacional de Salud" PATRIC_genome.csv | cut -f2 -d , | awk '{print $1,$2}' | sort | uniq -c

      1 Bartonella bacilliformis
      1 Genome Name
      3 Mycobacterium tuberculosis
      6 Neisseria meningitidis
      1 Salmonella enterica
      2 Vibrio parahaemolyticus
      1 Yersinia pestis


**04/2021**: I have uploaded most of the code used in this study. I hope this has been helpful for anyone who wants to check out this work.  